In [1]:
# %cd /notebooks/kaggle-garbage-classification
# !source .env3/bin/activate
# !pip install -U fastai
# !pip install -U fastcore

In [1]:
import torch
print(torch.__version__)  # Versão do PyTorch
print(torch.version.cuda)  # Versão do CUDA

2.5.1+cu118
11.8


In [2]:
from fastcore.xtras import Path
from fastai.vision.all import *
from PIL import Image

KeyboardInterrupt: 

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [5]:
root_path = Path("/notebooks/kaggle-garbage-classification")
data_path = Path(root_path/"data")
data_path.ls()

In [6]:
im_path = (data_path/'metal').ls()[0]
im = Image.open(im_path)
im

In [7]:
bad_imgs = []
classes = ['metal', 'trash', 'glass', 'cardboard', 'paper', 'plastic']
for c in classes:
    for im in (data_path/c).ls():
        try:
            _ = Image.open(im)
        except:
            bad_imgs.append(im)
            im.unlink()
bad_imgs, len(bad_imgs)

In [8]:
fnames = get_image_files(data_path)
fnames

## DataBlocks

In [9]:
data_block = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.)),
    batch_tfms=[*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)])

In [10]:
dls = data_block.dataloaders(data_path, bs=64, device=device)
dls

In [11]:
dls.vocab

In [12]:
dls.vocab.o2i

In [13]:
learn = vision_learner(dls, resnet34, metrics=error_rate, path=root_path)

In [14]:
#learn.lr_find()

In [15]:
learning_rate = 0.0006918309954

In [25]:
import ipynbname
notebook_name = ipynbname.name()
model_name = notebook_name + 'best_model'

In [27]:
from fastai.callback.tracker import SaveModelCallback

# Treinamento com salvamento automático dos melhores pesos
learn.fine_tune(
    50,  # Número máximo de épocas
    base_lr=3e-3,
    cbs=[
        SaveModelCallback(monitor='valid_loss', fname=model_name),  # Salva os melhores pesos baseados em 'valid_loss'
        EarlyStoppingCallback(monitor='valid_loss', patience=5)  # Early stopping opcional
    ]
)

In [28]:
# learn.path = root_path
# learn.path

In [29]:
#learn.save('fastai-Datablock')

In [30]:
learn.load(model_name, device=device)

## Predict

In [31]:
im_path = Path(root_path/'caixa-papelao-grande-50-c-x-26-l-x-26-a-kit-20-unidades-caixa-papelao-correio.jpg')
im = Image.open(im_path)
show_image(im); learn.predict(im_path)

In [32]:
learn.predict(im_path)

In [33]:
im_path = Path(root_path/'pote_vidro_500g_com_tampa_74_mm_preta.png')
im = Image.open(im_path)
show_image(im); learn.predict(im_path)